In [1]:
# import dependencies
import yfinance as yf
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta
import logging

In [ ]:
class YahooDataCollector:
    def __init__(self, db_config):
        self.db_config = db_config
        self.setup_logging()
    
    def setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('trading_system.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def connect_db(self):
        try:
            conn = mysql.connector.connect(**self.db_config)
            return conn
        except mysql.connector.Error as e:
            self.logger.error(f"Database connection error: {e}")
            return None
    
    def create_tables(self):
        """Create necessary database tables"""
        conn = self.connect_db()
        if not conn:
            return False
        
        cursor = conn.cursor()
        
        # Stock information table
        stock_table = """
        CREATE TABLE IF NOT EXISTS stocks (
            id INT AUTO_INCREMENT PRIMARY KEY,
            symbol VARCHAR(10) UNIQUE NOT NULL,
            company_name VARCHAR(255),
            sector VARCHAR(100),
            added_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        
        # Historical price data
        price_table = """
        CREATE TABLE IF NOT EXISTS historical_prices (
            id INT AUTO_INCREMENT PRIMARY KEY,
            symbol VARCHAR(10),
            date DATE,
            open_price DECIMAL(10,4),
            high_price DECIMAL(10,4),
            low_price DECIMAL(10,4),
            close_price DECIMAL(10,4),
            volume BIGINT,
            adj_close DECIMAL(10,4),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE KEY unique_symbol_date (symbol, date),
            FOREIGN KEY (symbol) REFERENCES stocks(symbol)
        )
        """
        
        # Simulated portfolio
        portfolio_table = """
        CREATE TABLE IF NOT EXISTS portfolio (
            id INT AUTO_INCREMENT PRIMARY KEY,
            symbol VARCHAR(10),
            shares_owned DECIMAL(10,4) DEFAULT 0,
            avg_cost_basis DECIMAL(10,4) DEFAULT 0,
            total_invested DECIMAL(10,2) DEFAULT 0,
            current_value DECIMAL(10,2) DEFAULT 0,
            unrealized_pnl DECIMAL(10,2) DEFAULT 0,
            last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
            UNIQUE KEY unique_symbol (symbol)
        )
        """
        
        # Simulated trades
        trades_table = """
        CREATE TABLE IF NOT EXISTS simulated_trades (
            id INT AUTO_INCREMENT PRIMARY KEY,
            symbol VARCHAR(10),
            action ENUM('BUY', 'SELL'),
            shares DECIMAL(10,4),
            price DECIMAL(10,4),
            total_value DECIMAL(10,2),
            commission DECIMAL(10,2) DEFAULT 0,
            trade_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            strategy VARCHAR(50),
            notes TEXT
        )
        """
        
        # Account balance tracking
        account_table = """
        CREATE TABLE IF NOT EXISTS account_balance (
            id INT AUTO_INCREMENT PRIMARY KEY,
            cash_balance DECIMAL(12,2) DEFAULT 100000.00,
            total_portfolio_value DECIMAL(12,2) DEFAULT 0,
            total_account_value DECIMAL(12,2) DEFAULT 100000.00,
            daily_pnl DECIMAL(10,2) DEFAULT 0,
            total_pnl DECIMAL(10,2) DEFAULT 0,
            date DATE UNIQUE,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        
        tables = [stock_table, price_table, portfolio_table, trades_table, account_table]
        
        try:
            for table in tables:
                cursor.execute(table)
            conn.commit()
            self.logger.info("Database tables created successfully")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error creating tables: {e}")
            return False
        finally:
            cursor.close()
            conn.close()
    
    def add_stock(self, symbol, company_name=None, sector=None):
        """Add a stock to tracking list"""
        conn = self.connect_db()
        if not conn:
            return False
        
        cursor = conn.cursor()
        
        # Get company info from Yahoo Finance if not provided
        if not company_name:
            try:
                stock = yf.Ticker(symbol)
                info = stock.info
                company_name = info.get('longName', symbol)
                sector = info.get('sector', 'Unknown')
            except:
                company_name = symbol
                sector = 'Unknown'
        
        try:
            query = """
            INSERT IGNORE INTO stocks (symbol, company_name, sector) 
            VALUES (%s, %s, %s)
            """
            cursor.execute(query, (symbol, company_name, sector))
            conn.commit()
            self.logger.info(f"Added stock: {symbol} - {company_name}")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error adding stock {symbol}: {e}")
            return False
        finally:
            cursor.close()
            conn.close()
    
    def fetch_historical_data(self, symbol, period="2y"):
        """Fetch historical data from Yahoo Finance"""
        try:
            stock = yf.Ticker(symbol)
            hist = stock.history(period=period)
            
            if hist.empty:
                self.logger.warning(f"No data found for {symbol}")
                return None
            
            # Reset index to get date as column
            hist.reset_index(inplace=True)
            hist['Symbol'] = symbol
            
            return hist
        except Exception as e:
            self.logger.error(f"Error fetching data for {symbol}: {e}")
            return None
    
    def store_historical_data(self, symbol, data):
        """Store historical data in database"""
        if data is None or data.empty:
            return False
        
        conn = self.connect_db()
        if not conn:
            return False
        
        cursor = conn.cursor()
        
        try:
            for _, row in data.iterrows():
                query = """
                INSERT IGNORE INTO historical_prices 
                (symbol, date, open_price, high_price, low_price, close_price, volume, adj_close)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """
                values = (
                    symbol,
                    row['Date'].date(),
                    float(row['Open']),
                    float(row['High']),
                    float(row['Low']),
                    float(row['Close']),
                    int(row['Volume']),
                    float(row['Close'])  
                )
                cursor.execute(query, values)
            
            conn.commit()
            self.logger.info(f"Stored {len(data)} records for {symbol}")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error storing data for {symbol}: {e}")
            return False
        finally:
            cursor.close()
            conn.close()
    
    def update_all_stocks(self):
        """Update historical data for all tracked stocks"""
        conn = self.connect_db()
        if not conn:
            return False
        
        cursor = conn.cursor()
        cursor.execute("SELECT symbol FROM stocks")
        symbols = [row[0] for row in cursor.fetchall()]
        cursor.close()
        conn.close()
        
        success_count = 0
        for symbol in symbols:
            self.logger.info(f"Updating data for {symbol}")
            data = self.fetch_historical_data(symbol)
            if self.store_historical_data(symbol, data):
                success_count += 1
        
        self.logger.info(f"Updated {success_count}/{len(symbols)} stocks")
        return success_count == len(symbols)
    
    def get_current_price(self, symbol):
        """Get current/latest price for a symbol"""
        try:
            stock = yf.Ticker(symbol)
            data = stock.history(period="1d", interval="1m")
            if not data.empty:
                return float(data['Close'].iloc[-1])
            else:
                # Fallback to daily data
                data = stock.history(period="5d")
                if not data.empty:
                    return float(data['Close'].iloc[-1])
            return None
        except Exception as e:
            self.logger.error(f"Error getting current price for {symbol}: {e}")
            return None
    
    def initialize_account(self, starting_balance=100000):
        """Initialize trading account with starting balance"""
        conn = self.connect_db()
        if not conn:
            return False
        
        cursor = conn.cursor()
        
        try:
            query = """
            INSERT IGNORE INTO account_balance 
            (cash_balance, total_portfolio_value, total_account_value, date)
            VALUES (%s, 0, %s, CURDATE())
            """
            cursor.execute(query, (starting_balance, starting_balance))
            conn.commit()
            self.logger.info(f"Initialized account with ${starting_balance:,.2f}")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error initializing account: {e}")
            return False
        finally:
            cursor.close()
            conn.close()

In [ ]:
if __name__ == "__main__":
    # Database configuration
    db_config = {
        'host': '127.0.0.1',
        'user': 'root',
        'password': '',
        'database': 'trading_system'
    }
    
    # Initialize collector
    collector = YahooDataCollector(db_config)
    
    # Create database tables
    collector.create_tables()
    
    # Initialize account with $100,000 virtual money
    collector.initialize_account(100000)
    
    # 20 validated stocks
    stocks_to_track = [
        ('KR', 'The Kroger Co.', 'Consumer Defensive'),           # Rank 1: 54.64%
        ('AAPL', 'Apple Inc.', 'Technology'),                     # Rank 2: 54.22%
        ('ERO', 'Ero Copper Corp.', 'Basic Materials'),           # Rank 3: 54.22%
        ('BLK', 'BlackRock, Inc.', 'Financial Services'),         # Rank 4: 53.59%
        ('RSP', 'Invesco S&P 500 Equal Weight ETF', 'ETF'),       # Rank 5: 53.16%
        ('JPM', 'JPMorgan Chase & Co.', 'Financial Services'),    # Rank 6: 52.95%
        ('FXP', 'ProShares UltraShort FTSE China 50', 'ETF'),     # Rank 7: 52.32%
        ('GOOGL', 'Alphabet Inc.', 'Communication Services'),     # Rank 8: 52.32%
        ('TSLA', 'Tesla, Inc.', 'Automotive'),                    # Rank 9: 51.90%
        ('MDT', 'Medtronic plc', 'Healthcare'),                   # Rank 10: 51.48%
        ('AMZN', 'Amazon.com, Inc.', 'Consumer Cyclical'),        # Rank 11: 51.05%
        ('VGK', 'Vanguard FTSE Europe ETF', 'ETF'),               # Rank 12: 50.63%
        ('OXY', 'Occidental Petroleum Corporation', 'Energy'),    # Rank 13: 50.42%
        ('PGJ', 'Invesco Golden Dragon China ETF', 'ETF'),        # Rank 14: 50.00%
        ('GXC', 'SPDR S&P China ETF', 'ETF'),                     # Rank 15: 49.79%
        ('NVDA', 'NVIDIA Corporation', 'Technology'),             # Rank 16: 49.58%
        ('SPY', 'SPDR S&P 500 ETF', 'ETF'),                       # Rank 17: 49.16%
        ('META', 'Meta Platforms, Inc.', 'Communication Services'), # Rank 18: 48.94%
        ('XPP', 'ProShares Ultra FTSE China 50', 'ETF'),          # Rank 19: 48.52%
        ('MSFT', 'Microsoft Corporation', 'Technology'),          # Rank 20: 47.68%
    ]
    
    for symbol, name, sector in stocks_to_track:
        collector.add_stock(symbol, name, sector)
    
    # Fetch and store historical data
    print("Fetching historical data... This might take a few minutes...")
    collector.update_all_stocks()
    
    print("Setup complete! Check your database for the data.")

2025-10-19 02:19:13,608 - INFO - Database tables created successfully
2025-10-19 02:19:13,654 - INFO - Initialized account with $100,000.00
2025-10-19 02:19:13,697 - INFO - Added stock: KR - The Kroger Co.
2025-10-19 02:19:13,754 - INFO - Added stock: AAPL - Apple Inc.
2025-10-19 02:19:13,818 - INFO - Added stock: ERO - Ero Copper Corp.
2025-10-19 02:19:13,876 - INFO - Added stock: BLK - BlackRock, Inc.
2025-10-19 02:19:13,937 - INFO - Added stock: RSP - Invesco S&P 500 Equal Weight ETF
2025-10-19 02:19:13,999 - INFO - Added stock: JPM - JPMorgan Chase & Co.
2025-10-19 02:19:14,087 - INFO - Added stock: FXP - ProShares UltraShort FTSE China 50
2025-10-19 02:19:14,167 - INFO - Added stock: GOOGL - Alphabet Inc.
2025-10-19 02:19:14,277 - INFO - Added stock: TSLA - Tesla, Inc.
2025-10-19 02:19:14,399 - INFO - Added stock: MDT - Medtronic plc
2025-10-19 02:19:14,580 - INFO - Added stock: AMZN - Amazon.com, Inc.
2025-10-19 02:19:14,680 - INFO - Added stock: VGK - Vanguard FTSE Europe ETF
20

Fetching historical data... This might take a few minutes...


2025-10-19 02:19:17,312 - INFO - Stored 502 records for AAPL
2025-10-19 02:19:17,314 - INFO - Updating data for AMD
2025-10-19 02:19:17,934 - INFO - Stored 502 records for AMD
2025-10-19 02:19:17,936 - INFO - Updating data for AMZN
2025-10-19 02:19:18,604 - INFO - Stored 502 records for AMZN
2025-10-19 02:19:18,606 - INFO - Updating data for BLK
2025-10-19 02:19:19,747 - INFO - Stored 502 records for BLK
2025-10-19 02:19:19,747 - INFO - Updating data for CHN
2025-10-19 02:19:20,552 - INFO - Stored 502 records for CHN
2025-10-19 02:19:20,554 - INFO - Updating data for ERO
2025-10-19 02:19:21,277 - INFO - Stored 502 records for ERO
2025-10-19 02:19:21,280 - INFO - Updating data for FXP
2025-10-19 02:19:22,232 - INFO - Stored 502 records for FXP
2025-10-19 02:19:22,234 - INFO - Updating data for GOOGL
2025-10-19 02:19:23,502 - INFO - Stored 502 records for GOOGL
2025-10-19 02:19:23,506 - INFO - Updating data for GXC
2025-10-19 02:19:24,778 - INFO - Stored 502 records for GXC
2025-10-19 02

Setup complete! Check your database for the data.


: 